In [38]:
import pandas as pd
import numpy as np

def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [12]:
train = pd.read_parquet('../raw/train.parquet')
train['monthyear'] = (pd.to_datetime(train['S_2']).dt.year.astype(str)+
                      pd.to_datetime(train['S_2']).dt.month.map("{:02}".format).astype(str))

l = train.groupby("customer_ID")['monthyear'].agg(['nunique'])

In [113]:
tm = pd.read_parquet('train_match.parquet').reset_index(drop=True)
tm = tm.merge(l, on='customer_ID')
tm['rownum'] = tm.groupby(['customer_ID']).cumcount()+1
tm['check'] = tm['nunique'] + tm['rownum']
tm = tm.drop('rownum', axis=1)

In [120]:
# tm[(tm['check']<=13)].drop(['nunique','check'], axis=1).to_parquet('trn_match.parquet')

In [114]:
tm[tm['customer_ID']=='0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a90d464dcf2bd5976fb']

,customer_ID,match_id,nunique,check
1981918,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,3cd01876f9f90b565a898068a31a73591ee11fd2ff89ce...,12,13
1981919,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,6921d678983ae3d289974978c30634491c606bc6c2a466...,12,14
1981920,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,6e02f263e9e776707ea78534b70354cf068ddad25318cf...,12,15
1981921,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,683df8bcd20b290f02f1891afbb2db46e7dec2f2fc353e...,12,16
1981922,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,1dc56dc44a2520849efcf7fd02dbcc5a40e84915154775...,12,17
1981923,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,1dc56dc44a2520849efcf7fd02dbcc5a40e84915154775...,12,18
1981924,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,e0f6bed67b4e9f7479c206e1bdcd8fdf6d4f971d5d41d5...,12,19
1981925,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,2ace77112efb046bbf3a7361b48b3fb12ab1b1c7919775...,12,20
1981926,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,6848ff93194a81eac219a6e601db451fda5e7401b8c962...,12,21
1981927,0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,6f471c27f41f566427903f92f174feefcfa5174824991b...,12,22


In [153]:
valid = pd.read_csv('../../../notebooks/oof_v2_dpv25fold_seed42.csv')
print(amex_metric(valid['target'], valid['prediction']))

0.7975376272588612


In [173]:
tm[(tm['check']<=2) & (tm['nunique']<=i)]

,customer_ID,match_id,nunique,check
0,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,1,2
12,000eee12732c9c7b790870c556bc33339d6d17725cbeb4...,6e7c9da9df5f51d5b250d22900f723d18811e0896bdf8f...,1,2
24,0015d621bacbfd59cc1479aea405645ecb480f6ee779e0...,8dc9d3c66bc0ee87755b9c3f3d040c15dd4b659f24dfad...,1,2
36,00163cf7b8cc9cb2781ac05331d83c3ae7720f4860ee8a...,878cab1ee4b0a5b1b28211a7740206e542a16ffc1f1bc0...,1,2
48,004bf15cf65c01b2173e53c5284933d41650e69d977ebd...,50819f5c2f3dac0898993cd3a2e5c0e3c3bd16a3e72d55...,1,2
...,...,...,...,...
61380,ffd968fe02b1dfa9e26e3369e36e2c5b67fa9d42f9f926...,d2e00586a349c4c4f5355e51b4538e7d247e792f72ceb2...,1,2
61392,ffde43969200715fd23e7ce84e39c93931143596250ac8...,9c8c44e61a43d6352210acad3da6599e75c401254fbadd...,1,2
61404,fff3fcf0e0c15a3069b7fba318b75c74c2729927a69cff...,00c519120ced9b343cbb912c9500c2536fc0b122dc6bfa...,1,2
61416,fff845a9c5e9206a7f783640085531e10d6a9e54c2f1cc...,094e4d5c50324c62c6b88d53923ebc5abeb12f4f2bfd29...,1,2


In [179]:
df = tm[(tm['check']<=13)].drop(['nunique','check'], axis=1).rename(columns={'customer_ID':'cid'}).merge(
    train, left_on='match_id', right_on='customer_ID')

,cid,match_id,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,monthyear
0,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2017-03-25,0.435670,0,0.006181,0.818240,0.005539,NaN,...,-1,-1,0,0,0.0,NaN,0,0.009784,0,201703
1,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2017-04-25,0.450918,0,0.003818,0.813252,0.005253,NaN,...,-1,-1,0,0,0.0,NaN,0,0.007115,0,201704
2,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2017-05-26,0.280560,0,0.008151,0.815121,0.003411,NaN,...,-1,-1,0,0,0.0,NaN,0,0.000317,0,201705
3,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2017-06-24,0.284312,0,0.008788,0.813490,0.005062,NaN,...,-1,-1,0,0,0.0,NaN,0,0.001519,0,201706
4,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2d28939d0ecfd5b0ae70fec74a407347eb16009def4126...,2017-07-26,0.435534,0,0.006814,0.811246,0.004217,NaN,...,-1,-1,0,0,0.0,NaN,0,0.008218,0,201707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647429,fff6b15fde729fbebdc690002c866e30e09b3c0e1b0aad...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,2017-11-06,1.009671,18,0.062356,0.810715,0.009890,0.136255,...,-1,-1,0,0,0.0,NaN,0,0.005824,0,201711
5647430,fff6b15fde729fbebdc690002c866e30e09b3c0e1b0aad...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,2017-12-01,0.976333,14,0.051773,1.003421,0.008595,0.132740,...,-1,-1,0,0,0.0,NaN,0,0.002121,0,201712
5647431,fff6b15fde729fbebdc690002c866e30e09b3c0e1b0aad...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,2018-01-06,0.970591,18,0.011962,1.001903,0.001009,0.124928,...,-1,-1,0,0,0.0,NaN,0,0.004116,0,201801
5647432,fff6b15fde729fbebdc690002c866e30e09b3c0e1b0aad...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,9ed627c0cab4bd97fdf84078eaa6b09b7a35c51e9f686e...,2018-02-17,0.993181,0,0.002464,1.009342,0.006871,0.129262,...,-1,-1,0,0,0.0,NaN,0,0.008476,0,201802


In [172]:
for i in range(1, 13):
    
    print(i)
    
    x = tm[(tm['check']<=2) & (tm['nunique']<=i)].merge(valid[['customer_ID','prediction']], 
                                                         left_on='match_id', right_on='customer_ID')
    x = x.drop(['nunique','customer_ID_y','check'], axis=1)
    x = x.rename(columns = {'customer_ID_x':'customer_ID'})

    y = x.groupby(['customer_ID'])['prediction'].mean().reset_index()
#     y = (0.5*x.groupby(['customer_ID'])['prediction'].last()+
#          0.5*x.groupby(['customer_ID'])['prediction'].mean()).reset_index()
    
    y = y.merge(valid[['customer_ID', 'target']], on='customer_ID')

#     nvalid = pd.concat([valid[~valid.customer_ID.isin(y.customer_ID)], 
#                y[['customer_ID','target','prediction']]], axis=0)
    
    k = valid.merge(y[['customer_ID','prediction']], on='customer_ID', how='left')
    k['prediction_y'] = np.where(k['prediction_y'].isna(), k['prediction_x'], k['prediction_y'])
    k['prediction'] = (1*k['prediction_x']+1*k['prediction_y'])/2.
    
    print(amex_metric(nvalid[nvalid.customer_ID.isin(tm[(tm['nunique']<=i)]['customer_ID'])]['target'],
                      nvalid[nvalid.customer_ID.isin(tm[(tm['nunique']<=i)]['customer_ID'])]['prediction']))
    print(amex_metric(valid[valid.customer_ID.isin(tm[(tm['nunique']<=i)]['customer_ID'])]['target'],
                      valid[valid.customer_ID.isin(tm[(tm['nunique']<=i)]['customer_ID'])]['prediction']))

1
0.5987603067629953
0.6225173217635962
2
0.58538181020927
0.6226079728530806
3
0.5932070590286949
0.6346767585761496
4
0.5855647965905878
0.6300117792947451
5
0.5912596706038201
0.6365798250458007
6
0.5939122016239926
0.6426331662786908
7
0.5982180594547802
0.6485652488655076
8
0.5948527026872668
0.6518806963951781
9
0.5947943545773008
0.6570058871126497
10
0.5902465555891765
0.659927197311278
11
0.5870601977938694
0.6631892002647499
12
0.5801976891282378
0.6719086819329969


1
0.5987603067629953
0.6225173217635962
2
0.58538181020927
0.6226079728530806
3
0.5932070590286949
0.6346767585761496
4
0.5855647965905878
0.6300117792947451
5
0.5912596706038201
0.6365798250458007
6
0.5939122016239926
0.6426331662786908
7
0.5982180594547802
0.6485652488655076
8
0.5948527026872668
0.6518806963951781
9
0.5947943545773008
0.6570058871126497
10
0.5902465555891765
0.659927197311278
11
0.5870601977938694
0.6631892002647499
12
0.5801976891282378
0.6719086819329969


In [117]:
for i in range(1, 13):
    
    print(i)
    
    x = tm[(tm['check']<=13) & (tm['nunique']<=i)].merge(valid[['customer_ID','prediction']], 
                                                         left_on='match_id', right_on='customer_ID')
    x = x.drop(['nunique','customer_ID_y','check'], axis=1)
    x = x.rename(columns = {'customer_ID_x':'customer_ID'})

    y = x.groupby(['customer_ID'])['prediction'].mean().reset_index()
    y = y.merge(valid[['customer_ID', 'target']], on='customer_ID')

#     nvalid = pd.concat([valid[~valid.customer_ID.isin(y.customer_ID)], 
#                y[['customer_ID','target','prediction']]], axis=0)
    
    k = valid.merge(y[['customer_ID','prediction']], on='customer_ID', how='left')
    k['prediction_y'] = np.where(k['prediction_y'].isna(), k['prediction_x'], k['prediction_y'])
    k['prediction'] = (k['prediction_y']+k['prediction_x'])/2.
    nvalid = k[['customer_ID','target','prediction']]


    print(amex_metric(nvalid['target'], nvalid['prediction']))

1
0.795724819322625
2
0.7934248663074812
3
0.7905368822623493
4
0.7873346191641379
5
0.7839593803578381
6
0.7798070615138606
7
0.7755586582263061
8
0.76971219568968
9
0.7630520034398819
10
0.7549213080908679
11
0.7470830262902379
12
0.7309965529376317


In [118]:
for i in range(1, 13):
    
    print(i)
    
    x = tm[(tm['check']<=13) & (tm['nunique']<=i)].merge(valid[['customer_ID','prediction']], 
                                                         left_on='match_id', right_on='customer_ID')
    x = x.drop(['nunique','customer_ID_y'], axis=1)
    x = x.rename(columns = {'customer_ID_x':'customer_ID'})

    y = (x.groupby(['customer_ID'])['prediction'].mean()+
         x.groupby(['customer_ID'])['prediction'].min()).reset_index()
    
#     y = x.groupby(['customer_ID'])['prediction'].mean().reset_index()
#     y = x.groupby(['customer_ID'])['prediction'].nlargest(13-i).reset_index().\
#     groupby(['customer_ID'])['prediction'].mean().reset_index()
    
    y = y.merge(valid[['customer_ID', 'target']], on='customer_ID')

#     nvalid = pd.concat([valid[~valid.customer_ID.isin(y.customer_ID)], 
#                y[['customer_ID','target','prediction']]], axis=0)
    
    k = valid.merge(y[['customer_ID','prediction']], on='customer_ID', how='left')
    k['prediction_y'] = np.where(k['prediction_y'].isna(), k['prediction_x'], k['prediction_y'])
    k['prediction'] = (k['prediction_y']+k['prediction_x'])/2.
    nvalid = k[['customer_ID','target','prediction']]

    print(amex_metric(nvalid['target'], nvalid['prediction']))

1
0.7959802738750741
2
0.793528618509592
3
0.7901046456974994
4
0.7869722879225409
5
0.7837356604493086
6
0.779757063741822
7
0.7754879223581244
8
0.7698665958899614
9
0.7632516628882402
10
0.7550993178966605
11
0.7470691591555402
12
0.7300072296742817
